In [90]:
# Importing the numpy library (for arrays operations)
# Importing the pandas library (for data analysis)
# Importing the matplotlib.pyplot interface (for MATLAB-like plots)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Dataset preparation
We start by loading the dataset (csv file) we will work with. We will use Netflix stock price from 01/01/2012 to 12/31/2021 downloaded from investing.com:<br>
https://www.investing.com/equities/netflix,-inc.-historical-data

In [91]:
df = pd.read_csv('NFLX Historical Data.csv')
df

,Date,Price,Open,High,Low,Vol.,Change %
0,"Dec 31, 2021",602.44,610.01,614.08,602.05,2.00M,-1.58%
1,"Dec 30, 2021",612.09,612.99,620.61,611.24,1.61M,0.25%
2,"Dec 29, 2021",610.54,610.71,613.98,604.68,1.28M,-0.03%
3,"Dec 28, 2021",610.71,614.95,618.41,609.69,1.85M,-0.39%
4,"Dec 27, 2021",613.12,615.00,615.00,609.25,1.86M,-0.16%
...,...,...,...,...,...,...,...
2513,"Jan 09, 2012",14.03,12.80,14.24,12.45,214.07M,13.79%
2514,"Jan 06, 2012",12.33,11.30,12.47,11.24,125.66M,8.83%
2515,"Jan 05, 2012",11.33,11.35,11.69,11.04,86.61M,-1.39%
2516,"Jan 04, 2012",11.49,10.34,11.57,10.26,100.55M,11.34%


For simplicity, we just keep the 'Open' values of the Netflix stock, and rechape the DataFrame 'df' into a vertical vector numpy array (2518 rows × 1 column).

In [92]:
arr = df['Open'].values
arr = arr.reshape(-1, 1)
arr.shape

(2518, 1)

We Split the data into a training set stored in an array "arr_train" and a testing set stored in an array "arr_test". The training set contains the first 80% of the data and the testing set contains the last 20% of the data.

In [95]:
arr_train = df[:int(arr.shape[0]*0.8)]
arr_test = df[int(arr.shape[0]*0.8):]
print(arr_train.shape)
print(arr_test.shape)

(2014, 7)
(504, 7)


# LSTM model

In [99]:
# Importing the estimator "MinMaxScaler" from the sklearn.preprocessing module (to scale data to a given range)
# See https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

from sklearn.preprocessing import MinMaxScaler